In [1]:
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
import pandas as pd
import numpy as np
import os
import cv2

# Checking model_x_sig.h5

In [2]:
labels = pd.read_csv('Data/train_label.csv')
sample = pd.read_csv('Data/sample_submission.csv')
# num_train_images = labels.shape[0]
num_train_images = 2520  # we choose 3300 images for this assignment. It works for a machine having 8Gb Ram. You can adjust it if your Ram is different. 
split_point = 2240 # split the data into training data [0:3000] and val data [3000:]
print('num_train_images:', num_train_images)
types = sample.columns[1:]
print('Types:', types)
num_classes = len(types)
print('num_classes:', num_classes)

num_train_images: 2520
Types: Index(['distribute', 'ineq', 'integral', 'limit', 'matrix', 'series', 'sqrt'], dtype='object')
num_classes: 7


In [3]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_width = 224

def get_image(filename):
    ########################################################################
    # TODO: Your code here...
    ########################################################################
    original = load_img(filename, target_size=(224,224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return image_batch[0]


In [4]:
x_train = np.zeros((num_train_images, img_width, img_width, 3), dtype=np.uint8)
y_train = np.zeros((num_train_images, num_classes), dtype=np.uint8)

count = 0

for i in range(num_train_images):
    x_train[i] = get_image('Data/train/%s.png' % labels['id'][i])
    pos_arrays = (types == labels['Type'][i]).nonzero() # recall that types is the array of classes
    pos = pos_arrays[0][0]
    y_train[i][pos] = 1
    count += 1
    if(count % 1000 == 0): print(count, 'images loaded')

1000 images loaded
2000 images loaded


In [5]:
from tensorflow.keras.applications import xception
xception_x_train = xception.preprocess_input(x_train.copy())
xception_model = xception.Xception(weights='imagenet', include_top=False)
xception_features = xception_model.predict(xception_x_train, batch_size=64, verbose=1)
print(xception_features.shape)

40/40 [==============================] - 196s 5s/step
(2520, 7, 7, 2048)


In [6]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2048)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2048)
(None, 7)


In [7]:
from tensorflow.keras.models import load_model
model = load_model("./models/model_x_sig.h5")

In [12]:
Test_prediction = []
for files in os.listdir('./Data/Test_4'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_4/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = xception.preprocess_input(image_batch)
        feature_input = xception_model.predict(image_batch)
        predictions = model.predict(feature_input)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)


In [13]:
y_test = pd.read_excel(r'./Data/test_4_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)

In [14]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.5714285714285714


In [17]:
for files in os.listdir('./Data/Test_4/'):
    
    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'Data/Test_4/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        image_batch = xception.preprocess_input(image_batch)
        feature_input = xception_model.predict(image_batch)
        predictions = model.predict(feature_input)
        pos = np.argmax(predictions)

        print('Image Name: ',files,' Prediction: ',types[pos])


Image Name:  dist.png  Prediction:  distribute
Image Name:  ineq.png  Prediction:  distribute
Image Name:  int.png  Prediction:  distribute
Image Name:  lim.png  Prediction:  limit
Image Name:  matrix.png  Prediction:  matrix
Image Name:  series.png  Prediction:  series
Image Name:  sqrt.png  Prediction:  distribute
